<a href="https://colab.research.google.com/github/ragnor-devop/ColabProduccion03/blob/main/Produccion03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [329]:
#pip install widgetsnbextension
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [330]:
import warnings 
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from ipywidgets import HTML

from IPython.display import display, clear_output

import time
import codecs
import io
import sys
import base64

In [331]:
#Librerias para metricas
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score

#Librerras para Preprocesado y holdout
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
#Librerias de los modelos a utilizar
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier
from sklearn.svm import SVC



In [332]:
#Librerias para manipulacion de datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [333]:

X_train=pd.read_csv("https://raw.githubusercontent.com/ragnor-devop/CreditosDataset01/main/DataSets/PruebaBorrar/Xtrain.csv")
y_train=pd.read_csv("https://raw.githubusercontent.com/ragnor-devop/CreditosDataset01/main/DataSets/PruebaBorrar/ytrain.csv")
X_test=pd.read_csv("https://raw.githubusercontent.com/ragnor-devop/CreditosDataset01/main/DataSets/PruebaBorrar/Xtest.csv")
y_test=pd.read_csv("https://raw.githubusercontent.com/ragnor-devop/CreditosDataset01/main/DataSets/PruebaBorrar/ytest.csv")

In [334]:

#Entrenamiento del modelo Logistic Regression
logistic_regresion = LogisticRegression(max_iter=10000)
lr_entrenamiento_tiempo_inicio= time.time()
logistic_regresion.fit(X_train, y_train)
lr_entrenamiento_tiempo_fin = time.time()
logistic_regression_tiempo_entrenamiento = lr_entrenamiento_tiempo_fin - lr_entrenamiento_tiempo_inicio # in seconds
# prediccion Logistic Regression
lr_predict_tiempo_inicio= time.time()
y_pred_regression = logistic_regresion.predict(X_test)
lr_predict_tiempo_fin = time.time()
logistic_regression_tiempo_prediccion = lr_predict_tiempo_fin - lr_predict_tiempo_inicio # in seconds
#Metricas Accuracy Score Logistic Regression
accuracy_logistic_regression = accuracy_score(y_test, y_pred_regression)

In [335]:


def generarBoton(res,estado):
  #FILE
  res=res.to_csv()
  filename = 'informe.csv'
  b64 = base64.b64encode(res.encode())
  payload = b64.decode()

  #BUTTONS
  html_buttons = '''<html>
  <head>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  </head>
  <body>
  
  <a download="{filename}" href="data:text/csv;base64,{payload}" download>
  <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Descargar Informe</button>
  </a>
  
  </body>
  </html>
  '''

  html_button = html_buttons.format(payload=payload,filename=filename)
  display(HTML(html_button))

In [336]:

def procesado(archivo_csv, estado):
  """
  "Realiza la preparacion de los datos"

  archivo_csv   = archivo en formato csv
  estado        = una cadena donde se registra el estado
  ## Retorno ##
  df      pandas.dataframe    conjunto de datos preprocesado
  estado  str                 
  
  """
  dfs=pd.DataFrame(data=[""],columns=[""])
  estado=""
  try:
    
    uploaded_file=archivo_csv.data[0]
    dfs=pd.read_csv(io.BytesIO(uploaded_file))
    

     #CODIGO PARA PREPROCESAR EL DATASET almacenado en df
    
    
  except Exception as ex:
    estado+="Archivo con formato incorrecto"
  dfp,estado = analisisDatos(dfs,estado)
  
  return dfp,estado


In [337]:
def analisisDatos(dfa, resultado):
  """
  El modelo analiza el conjunto de datos

  Parametros 
  df =        pandas.dataframe    El conjunto de datos dene estar procesado
  resultado = str                 Una cadena vacia

  Retorno

  df =        pandas.datafrana    El conjunto de datos con una columna adicional de valores predichos
  resultado   str                 Una cadena con el estado del proceso

  Errores
  SI el conjunto de datos no esta preprocesado
  """
  tiempo_final=0
  try:
    tiempo_inicio= time.time()
    prediccion=logistic_regresion.predict(dfa)
    tiempo_fin = time.time()
    tiempo_final= tiempo_fin - tiempo_inicio # in seconds

    dfa["Aprobar"]=prediccion
    dfa.loc[dfa["Aprobar"]==1,"Aprobar"]="Aprobar"
    dfa.loc[dfa["Aprobar"]==0,"Aprobar"]="No Aprobar"

    estado="Procesado Exitoso"
  except Exception as ex:
    resultado = resultado + "Error Archivo No valido" +"\n "

  return dfa,resultado
  

In [338]:
grand = widgets.ToggleButtons(
            options=['A', 'B']
        )

In [339]:
# name
# name = widgets.Text(placeholder='Your name here')
archivo=widgets.FileUpload(
    accept='.csv',
    multiple=False  # True to accept multiple files upload else False

)

In [340]:
date = widgets.DatePicker(description='Pick a Date')

In [341]:

# button send

button_send = widgets.Button(
                description='Enviar',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        res,estado = procesado(archivo,"")
        generarBoton(res,estado)
        

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])


In [342]:

# stacked right hand side

text_0 = widgets.HTML(value="<h1>Modelo de ML</h1>")
text_1 = widgets.HTML(value="<h2>Seleccionar tipo de archivo</h2>")
text_2= widgets.HTML(value="<h2>Agregar fecha de hoy </h2>")
text_3= widgets.HTML(value="<h2>Cantidad minima de registros</h2>")
text_4= widgets.HTML(value=f"<h2>Listo</h2>")


vbox_text = widgets.VBox([text_0, text_1, grand, archivo, text_2, date, text_3, text_4, text_5, vbox_result])

In [343]:
page = widgets.HBox([vbox_text])
display(page)

In [344]:
# !pip freeze > requirements.txt